# SPM Tutorial #6: Scripting

## Sections
- Creating the Template
- File Selection and File Splitting
- Filling in the Preprocessing Modules
- Editing the file
- Concatenating strings
- Loading the Onset Files
- Running the Script

## Notes
- Script targets all 26 subjects

In [ ]:
import pandas as pd
import numpy as np

print("\n" + "=" * 70)
print("TUTORIAL #6: BATCH SCRIPTING - AUTOMATION FRAMEWORK")
print("=" * 70)

def get_subject_list(dataset_path="ds000102", pattern="sub-*"):
    """Get list of subjects from dataset"""
    import glob
    subjects = sorted(glob.glob(f"{dataset_path}/{pattern}"))
    subjects = [s.split('/')[-1] for s in subjects]
    return subjects

def load_subject_data(subject_id, run_id=1, dataset_path="ds000102"):
    """Load behavioral and fMRI data for a subject/run"""
    events_file = f"{dataset_path}/{subject_id}/func/{subject_id}_task-flanker_run-{run_id}_events.tsv"
    bold_file = f"{dataset_path}/{subject_id}/func/{subject_id}_task-flanker_run-{run_id}_bold.nii.gz"
    try:
        events = pd.read_csv(events_file, sep="\t")
        return {'subject': subject_id, 'run': run_id, 'events': events, 'bold_file': bold_file}
    except FileNotFoundError:
        return None

def calculate_behavioral_metrics(events_df):
    """Calculate behavioral metrics from events dataframe"""
    metrics = {
        'n_trials': len(events_df),
        'accuracy': (events_df['correctness'] == 'correct').sum() / len(events_df),
        'mean_rt': events_df['response_time'].mean(),
        'std_rt': events_df['response_time'].std(),
        'congruent_rt': events_df[events_df['Stimulus'] == 'congruent']['response_time'].mean(),
        'incongruent_rt': events_df[events_df['Stimulus'] == 'incongruent']['response_time'].mean(),
    }
    metrics['flanker_effect'] = metrics['incongruent_rt'] - metrics['congruent_rt']
    return metrics

subject_list = get_subject_list()
print(f"\nDataset contains {len(subject_list)} subjects:")
print(f"  {', '.join(subject_list[:10])}{'...' if len(subject_list) > 10 else ''}")

print("\n" + "-" * 70)
print("BATCH PROCESSING SUMMARY")
print("-" * 70)

batch_results = []
for subject_id in subject_list[:5]:
    for run_id in [1, 2]:
        data = load_subject_data(subject_id, run_id)
        if data is not None:
            metrics = calculate_behavioral_metrics(data['events'])
            batch_results.append({
                'Subject': subject_id,
                'Run': run_id,
                'N_Trials': metrics['n_trials'],
                'Accuracy': f"{metrics['accuracy']:.1%}",
                'Mean_RT': f"{metrics['mean_rt']:.3f}s",
                'Flanker_Effect': f"{metrics['flanker_effect']*1000:.0f}ms",
            })

batch_df = pd.DataFrame(batch_results)
print(f"\n{batch_df.to_string(index=False)}")

print(f"\n✓ Batch processing framework ready")
print(f"✓ Total runs to process: {len(subject_list) * 2}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## Batch Processing Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Batch Processing Summary: First 5 Subjects', fontsize=14, fontweight='bold')

full_batch_results = []
for subject_id in subject_list[:5]:
    for run_id in [1, 2]:
        data = load_subject_data(subject_id, run_id)
        if data is not None:
            metrics = calculate_behavioral_metrics(data['events'])
            full_batch_results.append({
                'Subject': subject_id,
                'Run': run_id,
                'Accuracy': metrics['accuracy'],
                'Mean_RT': metrics['mean_rt'],
                'Flanker_Effect': metrics['flanker_effect'],
            })

full_batch_df = pd.DataFrame(full_batch_results)

ax = axes[0, 0]
subjects_unique = full_batch_df['Subject'].unique()
x_pos = np.arange(len(subjects_unique))
for run_id in [1, 2]:
    run_data = full_batch_df[full_batch_df['Run'] == run_id]
    acc_values = [run_data[run_data['Subject'] == s]['Accuracy'].values[0] if len(run_data[run_data['Subject'] == s]) > 0 else 0
                  for s in subjects_unique]
    ax.bar(x_pos + (run_id-1)*0.4, acc_values, width=0.4, label=f'Run {run_id}', alpha=0.8)

ax.set_xlabel('Subject', fontweight='bold')
ax.set_ylabel('Accuracy', fontweight='bold')
ax.set_title('Behavioral Accuracy Across Subjects', fontweight='bold')
ax.set_xticks(x_pos + 0.2)
ax.set_xticklabels(subjects_unique, rotation=45)
ax.set_ylim([0.8, 1.0])
ax.legend()
ax.grid(alpha=0.3, axis='y')

ax = axes[0, 1]
for run_id in [1, 2]:
    run_data = full_batch_df[full_batch_df['Run'] == run_id]
    rt_values = [run_data[run_data['Subject'] == s]['Mean_RT'].values[0] if len(run_data[run_data['Subject'] == s]) > 0 else 0
                 for s in subjects_unique]
    ax.bar(x_pos + (run_id-1)*0.4, rt_values, width=0.4, label=f'Run {run_id}', alpha=0.8)

ax.set_xlabel('Subject', fontweight='bold')
ax.set_ylabel('Mean Response Time (s)', fontweight='bold')
ax.set_title('Mean RT Across Subjects', fontweight='bold')
ax.set_xticks(x_pos + 0.2)
ax.set_xticklabels(subjects_unique, rotation=45)
ax.legend()
ax.grid(alpha=0.3, axis='y')

ax = axes[1, 0]
for run_id in [1, 2]:
    run_data = full_batch_df[full_batch_df['Run'] == run_id]
    flanker_values = [run_data[run_data['Subject'] == s]['Flanker_Effect'].values[0] * 1000 if len(run_data[run_data['Subject'] == s]) > 0 else 0
                      for s in subjects_unique]
    ax.bar(x_pos + (run_id-1)*0.4, flanker_values, width=0.4, label=f'Run {run_id}', alpha=0.8)

ax.set_xlabel('Subject', fontweight='bold')
ax.set_ylabel('Flanker Effect (ms)', fontweight='bold')
ax.set_title('Conflict Effect: Incongruent > Congruent', fontweight='bold')
ax.set_xticks(x_pos + 0.2)
ax.set_xticklabels(subjects_unique, rotation=45)
ax.axhline(0, color='k', linestyle='--', alpha=0.3)
ax.legend()
ax.grid(alpha=0.3, axis='y')

ax = axes[1, 1]
ax.violinplot([full_batch_df['Accuracy'].values], positions=[1], showmeans=True, showmedians=True)
ax.violinplot([full_batch_df['Mean_RT'].values * 1000], positions=[2], showmeans=True, showmedians=True)
ax.violinplot([full_batch_df['Flanker_Effect'].values * 1000], positions=[3], showmeans=True, showmedians=True)

ax.set_xticks([1, 2, 3])
ax.set_xticklabels(['Accuracy', 'Mean RT (ms)', 'Flanker Effect (ms)'])
ax.set_ylabel('Value', fontweight='bold')
ax.set_title('Behavioral Metrics Distribution', fontweight='bold')
ax.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('batch_processing_summary.png', dpi=100, bbox_inches='tight')
plt.show()

print("✓ Batch Processing Visualization Complete")